This Jupiter Notebook will be used to contain the Neighbourhood Selection project for the Coursera Data Science Capstone Project

In [1]:
import pandas as pd #Import Pandas with pd as alias.

In [2]:
import numpy as np #Import Numpy with np as alias.

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
